In [4]:
# Auto-install required packages
import sys
import subprocess

packages = {
    'fpdf': 'fpdf',
    'bs4': 'beautifulsoup4',
    'ollama': 'ollama-python',
    'requests': 'requests'
}

for module, package in packages.items():
    try:
        __import__(module)
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

print("✅ All packages installed!\n")

# Now import everything
import requests
from bs4 import BeautifulSoup
import ollama
from fpdf import FPDF
import re
from urllib.parse import urljoin, urlparse
import time

# ... rest of my code continues here ...

# STEP 3: Function to Get Website Content
# This is like a robot that visits a website and brings back the text
def get_website_content(url):
    """
    Visit a website and extract all the text content
    
    Simple explanation:
    - We go to the website
    - We read all the text
    - We bring it back in a clean format
    """
    try:
        print(f"🌐 Visiting website: {url}")
        
        # Set headers to pretend we're a web browser (some sites block robots)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        # Get the website
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Parse the HTML (convert website code to readable format)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements (we don't need JavaScript or CSS)
        for script in soup(['script', 'style', 'nav', 'footer']):
            script.decompose()
        
        # Get text
        text = soup.get_text()
        
        # Clean up: remove extra spaces and empty lines
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        print(f"✅ Successfully extracted {len(text)} characters from website")
        return text
    
    except Exception as e:
        print(f"❌ Error getting website: {e}")
        return None


# STEP 4: Function to Get Multiple Pages
# Some companies have important info on different pages
def get_multiple_pages(base_url, max_pages=3):
    """
    Get content from the main page and a few important sub-pages
    
    Simple explanation:
    - Start at the main page
    - Find links to other important pages (About, Services, etc.)
    - Visit those pages too
    - Combine all the information
    """
    all_content = []
    visited_urls = set()
    
    # Get main page content
    main_content = get_website_content(base_url)
    if main_content:
        all_content.append(f"MAIN PAGE:\n{main_content}")
        visited_urls.add(base_url)
    
    # Try to find and visit important pages
    try:
        response = requests.get(base_url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Look for important pages (about, services, products, team)
        important_keywords = ['about', 'services', 'products', 'team', 'company']
        links = soup.find_all('a', href=True)
        
        page_count = 1
        for link in links:
            if page_count >= max_pages:
                break
            
            href = link['href']
            full_url = urljoin(base_url, href)
            
            # Check if this is an important page and we haven't visited it
            if any(keyword in href.lower() for keyword in important_keywords):
                if full_url not in visited_urls and urlparse(full_url).netloc == urlparse(base_url).netloc:
                    print(f"📄 Found important page: {full_url}")
                    content = get_website_content(full_url)
                    if content:
                        all_content.append(f"\n\nADDITIONAL PAGE ({href}):\n{content}")
                        visited_urls.add(full_url)
                        page_count += 1
                        time.sleep(1)  # Be polite, wait 1 second between requests
    
    except Exception as e:
        print(f"⚠️ Could not fetch additional pages: {e}")
    
    # Combine all content
    combined = "\n\n".join(all_content)
    print(f"✅ Total content collected: {len(combined)} characters from {len(visited_urls)} pages")
    
    return combined


# STEP 5: Function to Ask Ollama to Analyze the Website
# This is where the AI reads the website and understands what the company does
def analyze_company_with_ollama(company_name, website_content, model="llama3.2"):
    """
    Use Ollama AI to understand the company and extract key information
    
    Simple explanation:
    - We give the website text to the AI
    - We ask specific questions about the company
    - The AI reads everything and gives us structured answers
    """
    print(f"🤖 Asking Ollama to analyze {company_name}...")
    
    # Limit content length to avoid overwhelming the AI
    max_content_length = 8000  # characters
    if len(website_content) > max_content_length:
        website_content = website_content[:max_content_length]
        print(f"⚠️ Content truncated to {max_content_length} characters")
    
    prompt = f"""
You are analyzing a company website to create a professional brochure.

Company Name: {company_name}

Website Content:
{website_content}

Please provide a structured analysis in the following format:

1. COMPANY OVERVIEW (2-3 sentences about what the company does)

2. MISSION & VISION (What is their purpose and goal?)

3. KEY PRODUCTS/SERVICES (List 3-5 main offerings)

4. TARGET AUDIENCE (Who are their customers? Clients? Investors? Recruits?)

5. COMPETITIVE ADVANTAGES (What makes them special or different?)

6. COMPANY VALUES (What principles do they operate by?)

7. KEY ACHIEVEMENTS (Any notable accomplishments, awards, or milestones?)

Be concise but informative. If information is not available, write "Not specified on website."
"""
    
    try:
        # Call Ollama
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        )
        
        analysis = response['message']['content']
        print("✅ Analysis complete!")
        return analysis
    
    except Exception as e:
        print(f"❌ Error with Ollama: {e}")
        return None


# STEP 6: Function to Generate Brochure Content
# Now we ask the AI to write the actual brochure text
def generate_brochure_content(company_name, analysis, model="llama3.2"):
    """
    Use the analysis to generate polished brochure content
    
    Simple explanation:
    - We take the analysis from step 5
    - We ask the AI to write it in a professional, marketing style
    - This becomes our brochure text
    """
    print("📝 Generating brochure content...")
    
    prompt = f"""
Based on this company analysis, create professional brochure content for {company_name}.

Analysis:
{analysis}

Write a compelling brochure with these sections:

1. HEADLINE (One powerful sentence that captures what they do)

2. ABOUT US (2-3 engaging paragraphs)

3. WHAT WE OFFER (Bullet points of services/products)

4. WHY CHOOSE US (3-5 compelling reasons)

5. OUR VALUES (Short paragraph)

6. CALL TO ACTION (Encouraging closing statement)

Write in a professional, engaging tone suitable for investors, clients, and potential recruits.
Use persuasive language but remain factual.
"""
    
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ]
        )
        
        brochure_content = response['message']['content']
        print("✅ Brochure content generated!")
        return brochure_content
    
    except Exception as e:
        print(f"❌ Error generating brochure: {e}")
        return None


# STEP 7: Function to Create PDF Brochure
# This takes our text and makes it into a pretty PDF file
class PDF(FPDF):
    """Custom PDF class with header and footer"""
    
    def __init__(self, company_name):
        super().__init__()
        self.company_name = company_name
    
    def header(self):
        """Header appears at the top of each page"""
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, self.company_name, 0, 1, 'C')
        self.ln(5)
    
    def footer(self):
        """Footer appears at the bottom of each page"""
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')
    
    def chapter_title(self, title):
        """Format for section titles"""
        self.set_font('Arial', 'B', 14)
        self.set_fill_color(200, 220, 255)
        self.cell(0, 10, title, 0, 1, 'L', 1)
        self.ln(4)
    
    def chapter_body(self, body):
        """Format for section content"""
        self.set_font('Arial', '', 11)
        # Handle special characters
        body = body.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 6, body)
        self.ln()


def create_pdf_brochure(company_name, brochure_content, website_url, filename="company_brochure.pdf"):
    """
    Create a beautiful PDF brochure
    
    Simple explanation:
    - Create a new PDF document
    - Add the company name at the top
    - Add all our brochure content with nice formatting
    - Add the website link
    - Save it as a PDF file
    """
    print(f"📄 Creating PDF brochure: {filename}")
    
    try:
        pdf = PDF(company_name)
        pdf.add_page()
        
        # Add brochure content
        pdf.set_font('Arial', '', 11)
        
        # Split content into sections and add to PDF
        sections = brochure_content.split('\n\n')
        
        for section in sections:
            if section.strip():
                # Check if this is a title (usually all caps or starts with number)
                if section.strip().isupper() or section[0].isdigit():
                    pdf.chapter_title(section.strip())
                else:
                    pdf.chapter_body(section.strip())
        
        # Add website at the end
        pdf.ln(10)
        pdf.set_font('Arial', 'I', 10)
        pdf.cell(0, 10, f'Visit us: {website_url}', 0, 1, 'C')
        
        # Save PDF
        pdf.output(filename)
        print(f"✅ PDF brochure created successfully: {filename}")
        return filename
    
    except Exception as e:
        print(f"❌ Error creating PDF: {e}")
        return None


# STEP 8: Main Function - Puts Everything Together!
# This is the master function that runs all the steps
def create_company_brochure(company_name, website_url, output_filename=None):
    """
    Main function that creates the entire brochure
    
    Simple explanation:
    - Step 1: Visit the website and collect information
    - Step 2: Analyze the information with AI
    - Step 3: Generate brochure content with AI
    - Step 4: Create a beautiful PDF
    - Step 5: Done! 🎉
    """
    print("\n" + "="*60)
    print(f"🚀 CREATING BROCHURE FOR: {company_name}")
    print("="*60 + "\n")
    
    # Set default filename if not provided
    if output_filename is None:
        output_filename = f"{company_name.replace(' ', '_')}_brochure.pdf"
    
    # STEP 1: Get website content
    print("STEP 1: Collecting website content...")
    website_content = get_multiple_pages(website_url, max_pages=3)
    
    if not website_content:
        print("❌ Failed to get website content. Aborting.")
        return None
    
    # STEP 2: Analyze with Ollama
    print("\nSTEP 2: Analyzing company with AI...")
    analysis = analyze_company_with_ollama(company_name, website_content)
    
    if not analysis:
        print("❌ Failed to analyze company. Aborting.")
        return None
    
    print("\n--- ANALYSIS ---")
    print(analysis)
    print("--- END ANALYSIS ---\n")
    
    # STEP 3: Generate brochure content
    print("STEP 3: Generating brochure content...")
    brochure_content = generate_brochure_content(company_name, analysis)
    
    if not brochure_content:
        print("❌ Failed to generate brochure content. Aborting.")
        return None
    
    print("\n--- BROCHURE CONTENT ---")
    print(brochure_content)
    print("--- END BROCHURE CONTENT ---\n")
    
    # STEP 4: Create PDF
    print("STEP 4: Creating PDF brochure...")
    pdf_file = create_pdf_brochure(company_name, brochure_content, website_url, output_filename)
    
    if pdf_file:
        print("\n" + "="*60)
        print("🎉 SUCCESS! Brochure created successfully!")
        print(f"📁 File saved as: {pdf_file}")
        print("="*60 + "\n")
        return pdf_file
    else:
        print("❌ Failed to create PDF. Please check errors above.")
        return None


# STEP 9: USAGE EXAMPLE
# This is how you actually use the brochure generator!

print("""
╔══════════════════════════════════════════════════════════════╗
║          COMPANY BROCHURE GENERATOR - READY TO USE!          ║
╔══════════════════════════════════════════════════════════════╗

HOW TO USE:
-----------
1. Make sure Ollama is running on your computer
2. Run this cell to create a brochure:

   create_company_brochure(
       company_name="Your Company Name",
       website_url="https://www.company-website.com"
   )

EXAMPLE:
--------
create_company_brochure(
    company_name="OpenAI",
    website_url="https://www.openai.com"
)

The brochure PDF will be saved in your current folder!
""")

# Uncomment the line below and add your company details:
# create_company_brochure("OpenAI", "https://www.openai.com")

✅ All packages installed!


╔══════════════════════════════════════════════════════════════╗
║          COMPANY BROCHURE GENERATOR - READY TO USE!          ║
╔══════════════════════════════════════════════════════════════╗

HOW TO USE:
-----------
1. Make sure Ollama is running on your computer
2. Run this cell to create a brochure:

   create_company_brochure(
       company_name="Your Company Name",
       website_url="https://www.company-website.com"
   )

EXAMPLE:
--------
create_company_brochure(
    company_name="OpenAI",
    website_url="https://www.openai.com"
)

The brochure PDF will be saved in your current folder!



In [5]:
create_company_brochure(
    company_name="Tesla",
    website_url="https://www.tesla.com"
)


🚀 CREATING BROCHURE FOR: Tesla

STEP 1: Collecting website content...
🌐 Visiting website: https://www.tesla.com
✅ Successfully extracted 202 characters from website
✅ Total content collected: 213 characters from 1 pages

STEP 2: Analyzing company with AI...
🤖 Asking Ollama to analyze Tesla...
✅ Analysis complete!

--- ANALYSIS ---
Here's the analysis of Tesla's company website:

1. COMPANY OVERVIEW:
Tesla, Inc. is an American multinational corporation primarily engaged in designing and manufacturing electric vehicles (EVs), solar energy systems, and energy storage products.

2. MISSION & VISION:
The mission statement on the website is not explicitly stated, but it can be inferred that Tesla aims to accelerate the world's transition to sustainable energy through its innovative products and services. The vision, however, is more abstract: "To accelerate the world's transition to renewable energy."

3. KEY PRODUCTS/SERVICES:

* Electric Cars
* Solar & Clean Energy systems
* Energy storag

'Tesla_brochure.pdf'

In [6]:
create_company_brochure("Google", "https://www.google.com")


🚀 CREATING BROCHURE FOR: Google

STEP 1: Collecting website content...
🌐 Visiting website: https://www.google.com
✅ Successfully extracted 120 characters from website
📄 Found important page: https://www.google.com/intl/ne/about.html
🌐 Visiting website: https://www.google.com/intl/ne/about.html
✅ Successfully extracted 1053 characters from website
✅ Total content collected: 1227 characters from 2 pages

STEP 2: Analyzing company with AI...
🤖 Asking Ollama to analyze Google...
✅ Analysis complete!

--- ANALYSIS ---
Here's the analysis of Google based on their company website:

**1. COMPANY OVERVIEW**

Google is a multinational technology company specializing in Internet-related services and products. They offer a wide range of products and services that include search engines, cloud computing, advertising technologies, software, and hardware.

**2. MISSION & VISION**

Not specified on the website, but according to their official website, Google's mission is "to organize the world's info

'Google_brochure.pdf'

In [7]:
create_company_brochure("Netflix", "https://www.netflix.com")


🚀 CREATING BROCHURE FOR: Netflix

STEP 1: Collecting website content...
🌐 Visiting website: https://www.netflix.com
✅ Successfully extracted 2737 characters from website
✅ Total content collected: 2748 characters from 1 pages

STEP 2: Analyzing company with AI...
🤖 Asking Ollama to analyze Netflix...
✅ Analysis complete!

--- ANALYSIS ---
Here's the analysis:

1. COMPANY OVERVIEW:
Netflix is a streaming service that offers a wide variety of award-winning TV shows, movies, anime, documentaries, and more on thousands of internet-connected devices.

2. MISSION & VISION:
The purpose of Netflix is to provide users with endless entertainment options while promoting binge-watching culture. The vision is to continuously update their content library with new titles and improve the user experience. Their mission is to give people "the power to watch what they want, when they want it."

3. KEY PRODUCTS/SERVICES:
- Unlimited movies and TV shows
- Streaming service available on various devices (sm

'Netflix_brochure.pdf'